In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/My Drive/class20211/PyTorch

/content/drive/My Drive/class20211/PyTorch


### 비디오 lab-10-1 (12분 근처) 명령프롬프트 창 따라하기

#### 예제 1

In [7]:
import torch
import torch.nn as nn

In [8]:
conv=nn.Conv2d(1,1,11,stride=4,padding=0)

In [9]:
conv

Conv2d(1, 1, kernel_size=(11, 11), stride=(4, 4))

In [10]:
inputs=torch.Tensor(1,1,227,227)
inputs.shape

torch.Size([1, 1, 227, 227])

In [11]:
out=conv(inputs)
out.shape

torch.Size([1, 1, 55, 55])

#### 예제 2

In [12]:
conv=nn.Conv2d(1,1,7,stride=2,padding=0)
inputs=torch.Tensor(1,1,64,64)
out=conv(inputs)
out.shape

torch.Size([1, 1, 29, 29])

#### 예제 3

#### 예제 4

#### 예제 5

### 비디오 lab-10-2 (2분 근처) 명령프롬프트 창 따라하기

In [13]:
import torch.nn as nn

In [14]:
inputs=torch.Tensor(1,1,28,28)

In [15]:
inputs.shape

torch.Size([1, 1, 28, 28])

In [16]:
conv1=nn.Conv2d(1,32,3,padding=1)
conv1

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [17]:
pool=nn.MaxPool2d(2)
pool

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

In [18]:
conv2=nn.Conv2d(32,64,3,padding=1)
conv2

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [19]:
out=conv1(inputs)
out.shape

torch.Size([1, 32, 28, 28])

In [20]:
out=pool(out)
out.shape

torch.Size([1, 32, 14, 14])

In [21]:
out=conv2(out)
out.shape

torch.Size([1, 64, 14, 14])

In [22]:
out=pool(out)
out.shape

torch.Size([1, 64, 7, 7])

In [23]:
out.size(0)

1

In [24]:
out.size(0)

1

In [25]:
out=out.view(out.size(0), -1)
out.shape

torch.Size([1, 3136])

In [26]:
fc=nn.Linear(3136,10)

In [27]:
out=fc(out)

In [28]:
out.shape

torch.Size([1, 10])

## 비디오 lab-10-2 (7분 부터)

In [29]:
# Lab 11 MNIST and Convolutional Neural Network
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.init


In [30]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [31]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [32]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [33]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [34]:
# CNN Model (2 conv layers)
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # L1 ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L2 ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # Final FC 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.fc(out)
        return out

In [35]:
# instantiate CNN model
model = CNN().to(device)

In [36]:
model

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=3136, out_features=10, bias=True)
)

In [37]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [38]:
# train my model
total_batch = len(data_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 0.220208153
[Epoch:    2] cost = 0.0608730577
[Epoch:    3] cost = 0.0460902005
[Epoch:    4] cost = 0.0365734324
[Epoch:    5] cost = 0.030059427
[Epoch:    6] cost = 0.0262925848
[Epoch:    7] cost = 0.0205167979
[Epoch:    8] cost = 0.0186392795
[Epoch:    9] cost = 0.0155233936
[Epoch:   10] cost = 0.0133829284
[Epoch:   11] cost = 0.0114775142
[Epoch:   12] cost = 0.00884278025
[Epoch:   13] cost = 0.00891989563
[Epoch:   14] cost = 0.00632858044
[Epoch:   15] cost = 0.00677486602
Learning Finished!


In [39]:
# Test model and check accuracy
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9842999577522278


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
